In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [13]:
#chargement des donnees
sales_df = pd.read_csv("/content/drive/MyDrive/Mes projets/Projet Prediction de la demande des articles dans un magasin/Datasets/Donnees bruts/ventes.csv")
stock_df = pd.read_csv("/content/drive/MyDrive/Mes projets/Projet Prediction de la demande des articles dans un magasin/Datasets/Donnees bruts/stocks.csv")
promotion_df = pd.read_csv("/content/drive/MyDrive/Mes projets/Projet Prediction de la demande des articles dans un magasin/Datasets/Donnees bruts/promotions.csv")
weather_df = pd.read_csv("/content/drive/MyDrive/Mes projets/Projet Prediction de la demande des articles dans un magasin/Datasets/Donnees bruts/meteo.csv")


1) Nettoyage des donnees
```
a.Vérification des valeurs manquantes
Nous devons identifier et traiter les valeurs manquantes dans chaque DataFrame.
```



In [14]:
missing_val_count_by_column_sales = (sales_df.isnull().sum())
missing_val_count_by_column_stock = (stock_df.isnull().sum())
missing_val_count_by_column_promotion = (promotion_df.isnull().sum())
missing_val_count_by_column_weather = (weather_df.isnull().sum())

# Vérification des valeurs manquantes
print("Valeurs manquantes dans les ventes:")
print(f"{sales_df.shape}\n{missing_val_count_by_column_sales[sales_df.isnull().sum() > 0]}")

print("\nValeurs manquantes dans les stocks:")
print(f"{stock_df.shape}\n{missing_val_count_by_column_stock[stock_df.isnull().sum() > 0]}")

print("\nValeurs manquantes dans les promotions:")
print(f"{promotion_df.shape}\n{missing_val_count_by_column_promotion[promotion_df.isnull().sum() > 0]}")

print("\nValeurs manquantes dans la météo:")
print(f"{weather_df.shape}\n{missing_val_count_by_column_weather[weather_df.isnull().sum() > 0]}")

Valeurs manquantes dans les ventes:
(21619, 7)
Quantité vendue    4265
dtype: int64

Valeurs manquantes dans les stocks:
(200, 5)
Stock actuel                    14
Seuil de réapprovisionnement    10
dtype: int64

Valeurs manquantes dans les promotions:
(56, 7)
ID promotion    2
dtype: int64

Valeurs manquantes dans la météo:
(3660, 4)
Température (°C)       785
Précipitations (mm)    933
dtype: int64




```
b. Traitement des valeurs manquantes
La stratégie de traitement inclus :
    Suppression des lignes avec des valeurs manquantes.
    Remplacement par la moyenne, la médiane.
    

```



 Supprimer les lignes avec des valeurs manquantes dans les promotions et les ventes

In [15]:
#Obtention des colonnes avec des donnes manquantes dans les ventes et les promotions
col_with_missing_sales=[col for col in sales_df.columns if sales_df[col].isnull().any()]
col_with_missing_promotions=[col for col in promotion_df.columns if promotion_df[col].isnull().any()]
#Supression dans les ventes
sales_df.dropna(subset=col_with_missing_sales, inplace=True)
#Suppression dans les promotions
promotion_df.dropna(subset=col_with_missing_promotions, inplace=True)

Remplace les valeurs manquantes par la température moyenne, pour la météo

In [16]:
#Obtention des colonnes avec des donnes manquantes dans la meteo
col_with_missing_weather=[col for col in weather_df.columns if weather_df[col].isnull().any()]

#Remplacement par la moyenne
for column in col_with_missing_weather:
  weather_df.fillna({column:weather_df[column].mean()}, inplace=True)

Remplacer les valeures manquante dans les stocks par la mediane

In [17]:
#Obtention des colonnes avec des donnes manquantes dans les stocks
col_with_missing_stock=[col for col in stock_df.columns if stock_df[col].isnull().any()]

#Remplacement par la moyenne
for column in col_with_missing_stock:
  stock_df.fillna({column:stock_df[column].median()}, inplace=True)

In [8]:
stock_df.describe()

,Stock actuel,Seuil de réapprovisionnement
count,200.000000,200.000000
mean,309.375000,92.025000
std,99.632122,8.804086
min,135.000000,78.000000
25%,229.500000,84.000000
50%,301.500000,92.000000
75%,383.250000,99.250000
max,487.000000,108.000000


In [9]:
weather_df.describe()

,Température (°C),Précipitations (mm)
count,3660.000000,3660.000000
mean,12.358226,49.692666
std,11.494542,24.940090
min,-10.000000,0.000000
25%,4.300000,32.875000
50%,12.358226,49.692666
75%,20.600000,66.100000
max,35.000000,100.000000


In [10]:
promotion_df.describe()

,Réduction (%)
count,54.000000
mean,28.333333
std,12.813613
min,5.000000
25%,18.500000
50%,28.500000
75%,38.750000
max,49.000000


In [11]:
sales_df.describe()

,Quantité vendue,Prix unitaire
count,17354.000000,17354.000000
mean,297.501383,99.980304
std,156.387834,50.403932
min,105.000000,12.880000
25%,185.000000,56.052500
50%,266.000000,100.060000
75%,349.000000,143.930000
max,875.000000,187.450000




```
c. Identification et traitement des outliers
Nous devons détecter et traiter les outliers, en particulier dans les colonnes numériques.
```



In [18]:
# Détection d'outliers pour 'Quantité vendue'
Q1 = sales_df['Quantité vendue'].quantile(0.25)
Q3 = sales_df['Quantité vendue'].quantile(0.75)
IQR = Q3 - Q1

#Definir les limites pour consider un outlier
lower_bound = Q1 - 0.4 * IQR
upper_bound = Q3 + 0.6 * IQR
print(lower_bound)
print(upper_bound)
#Filtrer les outliers
sales_df = sales_df[(sales_df['Quantité vendue'] >= lower_bound) & (sales_df['Quantité vendue'] <= upper_bound)]

119.39999999999999
447.4


Sauvegarde des donnees nettoyees

In [19]:
# Sauvegarde dans un fichier CSV
sales_df.to_csv("/content/drive/MyDrive/Mes projets/Projet Prediction de la demande des articles dans un magasin/Datasets/Donnees nettoyees/ventes_nettoyees.csv", index=False)
stock_df.to_csv("/content/drive/MyDrive/Mes projets/Projet Prediction de la demande des articles dans un magasin/Datasets/Donnees nettoyees/stocks_nettoyees.csv", index=False)
promotion_df.to_csv("/content/drive/MyDrive/Mes projets/Projet Prediction de la demande des articles dans un magasin/Datasets/Donnees nettoyees/promotions_nettoyees.csv", index=False)
weather_df.to_csv("/content/drive/MyDrive/Mes projets/Projet Prediction de la demande des articles dans un magasin/Datasets/Donnees nettoyees/meteo_nettoyees.csv", index=False)